# Семинар 3. Исправление опечаток

In [ ]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Недостатком тут является то, что не всегда можно правильно сопоставить слова правильного предложения и ошибочного (из-за слов с пропущенным или добавленным пробелом). Из статьи авторов корпуса не очень понятно, как они решали эту проблему, поэтому я просто удалил все такие предложения.

In [ ]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [ ]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

In [ ]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [ ]:
pprint(align_words(true[1], bad[1]))

Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [ ]:
mistakes = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        
        total += 1

In [ ]:
print('Доля ошибок - ', len(mistakes)/total )

In [ ]:
mistakes[:5]

Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [ ]:
Counter(mistakes).most_common(10)

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Новостные тексты для этого хорошо подходят.

Я заранее собрал небольшой корпус новостных текстов и немного почистил его удалив отдельную пунктуацию и пунктуацию на границах слов.

In [ ]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()

In [ ]:
# нормализация нам тут не нужна так как нужно находить слова в разных формах
print(corpus[1].split()[:10])

## Задание 1.
Напишите функцию, которая будет предсказывать ошибочные слова на основе корпуса.

In [ ]:
# создайте множество, чтобы проверять вхождения
vocab = set()

## ваш код здесь

In [ ]:
def predict_mistaken(word, vocab):
    '''
    ::input: word, vocabulary
    ::output: 1 or 0
    
    '''
    ## ваш код здесь
    
    

In [ ]:
# для оценки создайте два списка y_true и y_pred
# пройдитесь по предложениям
# сопоставите слова с помощью функции align_words
# пройдитесь по парам слов и
# если слова одинаковые добавьте в y_true 0 
# если слова разные добавьте в y_true 1
# предскажите ошибочность слова из bad списка 
# добавьте предсказание в список y_pred

y_true = []
y_pred = []

## ваш код здесь
    

In [ ]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred))

### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известной реализации Питера Норвига.

Основная идея - сделать словарь правильных слов (у нас уже есть), расчитать вероятность каждого слова в корпусе.

In [ ]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [ ]:
WORDS.most_common(10)

In [ ]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [ ]:
P('апофеоз')

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [ ]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
%%time
correction('опефеоз')

Давайте поподробнее разберем, что происходит в функции edits.

In [ ]:
word = 'опефеоз'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [ ]:
splits[:10]

In [ ]:
deletes = [L + R[1:] for L, R in splits if R]

In [ ]:
deletes[:10]

In [ ]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [ ]:
transposes[:10]

In [ ]:
letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [ ]:
len(replaces)

In [ ]:
inserts = [L + c + R for L, R in splits for c in letters]

In [ ]:
inserts[:10]

Для оценки используем просто долю правильных исправлений.

In [ ]:
# До этого мы уже считали долю ошибок во всех предложениях.
# Поэтому если ничего не менять то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Наш подход соответственно должен показывать лучший результат 
correct = 0
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = correction(pair[1])
        if predicted == pair[0]:
            correct += 1
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

In [ ]:
print(correct/total)

In [ ]:
%%time
correction('нав')

In [ ]:
%%time
correction('насмехатьсяаававттававаываываы')

Посмотрим какие исправления выбираются для самых частотных опечаток.

In [ ]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

In [ ]:
# в питоне есть библиотеке для нахождения близких строк

In [ ]:
from difflib import get_close_matches

In [ ]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

Работает тоже не очень быстро.

Недавно вышла библиотека, в которой реализованы многие методы нахождения расстояний.

In [27]:
!pip install textdistance

In [ ]:
import textdistance


In [ ]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [ ]:
%%time
get_closest_match_with_metric('опофиоз', WORDS, textdistance.hamming)

In [ ]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

Ждать так долго мы не можем, поэтому попробуем что-то побыстрее.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово.

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [ ]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [ ]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [ ]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [ ]:
%%time
get_closest_match_vec('опофеоз', X, vec)

## Задание 2. 


Напишите функцию, которая принимает слово и находит ближайшее к нему в словаре (сгенерируйте несколько кандидатов с помощью get_closest_match_vec, а затем посчитайте метрики близости до кадого слова и выведете самое близкое). 

In [ ]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    candidates = get_closest_match_vec(text, X, vec, TOPN=5)
    closest = candidates[0]
    dist_dict = []
    for candidate in candidates:
        dist_dict.append([candidate, metric(text, candidate)])
    minimal = dist_dict[0][1]
    closest = dist_dict[0][0]
    for variant in dist_dict:
        if variant[1] < minimal:
            minimal = variant[1]
            closest = variant[0]
    return closest

In [ ]:
get_closest_hybrid_match('локагль', X, vec)

ну мне кажется, что здесь нормально посчитать accuracy в лоб

In [ ]:
def accuracy(pred, test):
    right = 0
    for i in range(len(test)):
        if pred[i] = test[i]:
            right += 1
    return right / len(pred)

Но так, как (наверное))) было задумано выше, я тоже попыталась сделать

In [ ]:
vocab = [word for text in corpus for word in text.split()]

y_true = []
y_pred = []

for i in range(len(true)):
    for j in range(len(true[i].split())):
        pred = get_closest_hybrid_match(bad[i].split()[j],X,vec)
        if true[i][j] == pred:
            y_true.append(0)
        else:
            y_true.append(0 if pred in vocab else 1)
    print('text {} done'.format(i))
print(classification_report(y_true, y_pred))

In [ ]:
bad = [[word for word in text.split()] for text in bad]
true = [[word for word in text.split()] for text in true]
print(accuracy(true, [[get_closest_hybrid_match(word) for word in text] for text in bad]))

У меня что-то с юпайтером и я не могу проверить рабочесть кода, но кажется, что должно

В рассмотренных методах при выборе исправления никак не использовался контекст. Про то как это можно сделать, можно почитать вот тут - https://habr.com/ru/post/346618/